# Import Dependencies

In [ ]:
!apt-get install -y xvfb python3-opengl ffmpeg > /dev/null 2>&1
!pip install gym[classic_control]
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup gym pyvirtualdisplay tensorflow ffmpeg imageio-ffmpeg > /dev/null 2>&1

The action is an ndarray with shape (1,) which can take values {0, 1} indicating pushing the cart to the left or right, respectively. Note that the velocity that is reduced or increased by the applied force is not fixed and it depends on the angle the pole is pointing. The center of gravity of the pole varies the amount of energy needed to move the cart underneath it.

In [ ]:
import math
from typing import Dict, List, Tuple
from abc import ABC, abstractmethod
from gym import make
import numpy as np
from gym.wrappers.record_video import RecordVideo
import pickle
from typing import NamedTuple
from collections import defaultdict

# Load Cartpole Environment

We define a custom `CartPoleWorld` class to represent the cartpole environment. 

In [ ]:
'''
Update CartPoleWord class here
'''   
class CartpoleWorld():
    def __init__(self, display: bool = False) -> None:
        if display:
            self.__env = make("CartPole-v1", render_mode="human")
        else:
            self.__env = make("CartPole-v1")
        self.__observation: np.ndarray
        self.__reward: float = 0
        self.__truncated: bool  = False
        self.__done: bool = False
        self.__observation, _ = self.__env.reset()
    
    def get_actionspace(self):
        return self.__env.action_space
        
    def get_observation(self) -> np.ndarray:
        return self.__observation
    
    def update_world(self,action) -> float:
        self.__observation, self.__reward, self.__truncated, self.__done, _ = self.__env.step(action)
        return self.__reward
    
    def isEnd(self) -> bool:
        # position range
        if not (-2.4 < self.__observation[0] < 2.4):
            return True
        # angle range
        if not (-.2095 < self.__observation[2] < .2095):
            return True
        return self.__done or self.__truncated
    
    def get_reward(self) -> float:
        return self.__reward
    
    def resetWorld(self) -> Observation:
        self.__observation, _ = self.__env.reset()
        self.__reward = 0
        self.__done = False
        self.__truncated  = False
        return Observation(*self.__observation)
        
    def set_to_display_mode(self) -> None:
        self.__env = make("CartPole-v1", render_mode="human")
        self.__observation, _ = self.__env.reset()
        
    def set_save_video(self) -> None:
        self.__env  = make("CartPole-v1", render_mode="rgb_array_list")
        self.__env  = RecordVideo(self.__env , video_folder="video", name_prefix = "rl-video")
        
world = CartpoleWorld()

Check if there are 2 valid discrete actions that can be performed

In [ ]:
world.get_actionspace()

Display observable space. Note the format of the space being
{
    position,
    velocity,
    pole angle,
    pole angular velocity,
}

In [ ]:
world.get_observation()

In [ ]:
world.resetWorld()
print("Initial observations:", world.get_observation())

# Task 1

Development of an RL agent. Demonstrate the correctness of the implementation by sampling a random state from the cart pole environment, inputting to the agent, and outputting a chosen action. Print the values of the state and chosen action in Jupyter notebook.

### Observation
We define an `Observation` class as a `NamedTuple`, allowing us to retrive the value by index and by name

In [ ]:
class Observation(NamedTuple):
    d: float
    x: float
    theta: float
    omega: float
        

### Base RL Agent

We define an abstract base RL agent as follows:

In [ ]:
class RLAgent(ABC):
    def __init__(self, env:CartpoleWorld) -> None:
        self._env = env
        self._total_reward: float = 0
        
    @abstractmethod
    def get_optimal_action(self, s: Observation) -> int:
        pass
    
    def move(self, state: Observation) -> float:
        if (self._env.isEnd()):
            raise Exception("Episode already terminated")
        action = self.get_optimal_action(state)
        reward = self._env.update_world(action)
        # update reward
        self._total_reward += reward
        return reward
    
    
    def run_training(self, num_of_episode: int) -> None:
        pass
    
    def run_single_episode_training(self) -> int:
        pass

    @abstractmethod
    def run_production(self, num_of_episode: int) -> None:
        pass
    
    @abstractmethod
    def run_single_episode_production(self) -> int:
        pass
    
    def wrap_observation(self, observation: np.ndarray) -> Observation:
        """Converts numpy array to Observation object

        Args:
            observation (np.ndarray): array to pass in from cartpole

        Returns:
            Observation: Object to return
        """
        return Observation(*observation)
    
    def discretise_observation(self, observation: np.ndarray) -> Observation:
        # Position round off to 0.1 precision
        # Velocity round off to whole
        # Angle round off to 0.01 precision
        # Velocity round off to whole
        observation[0] = round(observation[0],1)
        observation[1] = round(observation[1],0)
        observation[2] = round(observation[2],2)
        observation[3] = round(observation[3],0)
        return Observation(*observation)
    

### 1a) Monte Carlo Agent

We first attempt to implement a Monte Carlo agent. Being the very first topic introduced in the lecture of Reinforcement Learning, it is also one of the easiest to implement. We use this agent as a baseline performance measure.

In [ ]:
'''
MC Agent Here
'''
class MCAgent():
    def __init__(self, env: CartpoleWorld) -> None:
        self.env = env
        self.total_reward = 0
        self.actions = [0,1]
        self.exploit_rate = 0
        self.gamma = 0.9
        self.actual = False

        # Format: dict[(state,action)] = [count, score]
        self.Q = defaultdict(lambda: [0, 0])

    def discretise_observation(self, observation: np.ndarray) -> Observation:
        # Position round off to 0.1 precision
        # Velocity round off to whole
        # Angle round off to 0.01 precision
        # Velocity round off to whole
        observation[0] = round(observation[0], 1)
        observation[1] = round(observation[1], 0)
        observation[2] = round(observation[2], 2)
        observation[3] = round(observation[3], 0)
        return Observation(*observation)
    
    def geometric_progression(self, n: int) -> float:
        # Calcuate rewards based on the formula S = a(1 - r^n) / 1 - r
        return (1 - (self.gamma ** n)) / (1 - self.gamma)


    def run_single_episode(self) -> int:
        # clear history
        self.env.resetWorld()
        self.total_reward = 0

        history = []

        while (not self.env.isEnd()):
            ndarry: np.ndarray = self.env.get_observation()
            s = Observation(*self.discretise_observation(ndarry))

            action = self.get_optimal_action(s)
            reward = self.env.update_world(action)
            history.append((s, action))
            # update reward
            self.total_reward += reward

        self.update_q_table(history)
        #self.max_reward = max(self.max_reward, self.total_reward)
        return self.total_reward
    

    def update_q_table(self, history: List[Tuple[Observation, int]]) -> None:
        state_action_pair = defaultdict(int)
        # Adopts first visit by updating from the back
        # Hence, only the first occurence of each (state,action) pair will be recorded
        # n is the episode length starting from the back, which is also the reward 
        for n,val in enumerate(history[::-1], start=1):
            state_action_pair[val] = n

        for key,val in state_action_pair.items():
            count, score = self.Q[key]
            score = (count * score + val) / (count + 1)
            self.Q[key] = [count+1, score]


    def get_optimal_action(self, s: Observation):
        
        #Slowly increase the exploitation rate, such that the agent will choose the best action as time progresses
        if not self.actual and random.random() >= self.exploit_rate:
            #return random.randint(0, 1)
            
            action = 1 if s[2] > 0 else 0
            # Chooses the physics action which a probability of 0.6, and the other action with a prob of 0.4
            return action if random.random() <= 0.60 else 1 - action
        
        return max(self.actions, key=lambda a: self.Q[(s,a)])
    

    def run(self, num_episodes: int, display: bool = False) -> None:
        cumulated_reward = 0

        # Pre-training, maybe can replace with pre-load data here in the future
        # Run 1000 times first using the physics solution to populate values
        self.exploit_rate = 0.0
        for _ in range(1000):    
            self.run_single_episode()


        outer_range, inner_range = 10, 5000
        self.exploit_rate = 0.7

        for _ in range(outer_range):
            total = 0
            max_reward = 0
            for _ in range(inner_range):
                # Using a variable exploitation rate, dk if its a good idea
                #self.exploit_rate = i / inner_range
                reward = self.run_single_episode()
                total += reward
                max_reward = max(max_reward, reward)
                

            print(f"Mean reward is: {total / inner_range}")
            print(f"Max reward is: {max_reward}")

        # Actual run
        self.actual = True

        if display:
            self.env.set_to_display_mode()

        for _ in range(num_episodes):
            current_reward = self.run_single_episode()
            cumulated_reward += current_reward

mc_agent = MCAgent(world)

## Explanation of Monte Carlo Agent

### <span style="color:red">ToDo:</span>
 
  - Find stuff talking about MC sampling
  - Motivation for using a hybrid initial policy (as compared to pure random sampling)
  - Motivation for using a decaying epsilon rate
  - How does the agent obtain the optimal action
  - How does the agent update the q-table

Correctness of implementation

In [ ]:
observation = world.resetWorld()
action = mc_agent.get_optimal_action(observation)
print(f"{observation}")
print(f"Chosen Action: {action}")

### 1b) Q-Learning Agent

The second agent we implemented was a Q-Learning Agent. Being the second topic taught in the lecture, it is an off-policy reinforcement learning that will find the best course of action, given the current state of the agent. 

In [ ]:
'''
Q-learning Agent here
'''
class QLearningAgent(RLAgent):
    def __init__(self, env:CartpoleWorld) -> None:
        super().__init__(env)
        
        # defines learning rate
        self.__learning_rate = 0.5
        self.__learning_rate_min = 0.2
        
        # defined for epsilon soft policy
        # initially set to a large number to encourage exploration
        # epsilon will decay as episodes increase
        self.__epsilon = 0.9
        self._epsilon_min = 0.1
        
        # dictionary of (state,action) -> quality
        self.__q_table : Dict[Tuple[Observation,int],float] = dict()
        self.__pi_table : Dict[Observation, int] = dict()
        
        # Load pickle file to restore agent previous state
        self.load_pickle('QL_parameters.pkl')
        
        # [left, right] action set
        self.__actions = [0,1]
        self.__discounted_reward = 0.9
        
        # parameter for production
        self.__is_production = False
        
    
    def get_optimal_action(self, s: Observation):
        # a* is the argmax_a Q(s,a)
        a_star: int = self.argmax_a_Q(s,self.__actions)
        
        if (self.__is_production):
            return a_star
        
        epsilon_over_A: float = self.__epsilon / len(self.__actions)
        
        # apply epsilon soft policy here to encourage exploration
        if (np.random.randn() < 1 - self.__epsilon + epsilon_over_A):
            # pick optimal
            self.__pi_table[s] = a_star
        else:
            # pick random
            self.__pi_table[s] = self.get_random_action()
        return self.__pi_table[s]
    
    def update_parameters(self) -> None:
        self.decay_epsilon()
        self.decay_learning_rate()
    
    def decay_epsilon(self) -> None:
        if self.__epsilon <= self._epsilon_min:
            return
        self.__epsilon *= 0.999999
    
    def decay_learning_rate(self) -> None:
        if self.__learning_rate <= self.__learning_rate_min:
            return
        self.__learning_rate *= 0.9999999
    
    def run_training(self,  num_of_episode: int):
        """Overrides base class method

        Args:
            num_of_episode (int): Number of episdoe to run
        """
        cumulated_reward = 0
        for _ in range(num_of_episode):
            cumulated_reward += self.run_single_episode()
        
        print(f"Epsilon: {self.__epsilon}, Discounted reward: {self.__discounted_reward}, Learning rate: {self.__learning_rate}")
        print(f"Mean reward is: {cumulated_reward/num_of_episode} for {num_of_episode} episodes")
        self.save_pickle("QL_parameters.pkl")
    
    def run_production(self, num_of_episode: int):
        self.__is_production = True
        
        cumulated_reward = 0
        for _ in range(num_of_episode):
            cumulated_reward += self.run_single_episode_production()

        print(f"Mean reward is: {cumulated_reward/num_of_episode} for {num_of_episode} episodes")

    def run_single_episode_training(self) -> int:
        # clear history
        self._env.resetWorld()
        self._total_reward = 0
        
        s_prime = self._env.get_observation()
        s_prime = self.discretise_observation(s_prime)
        
        while (not self._env.isEnd()):
            s = s_prime
            R = self.move(s)
            s_prime = self._env.get_observation()
            s_prime = self.discretise_observation(s_prime)
            
            self.update_q_table(s,R,s_prime)
            self.update_parameters()
        return self._total_reward
    
    def run_single_episode_production(self) -> int:
        # clear history
        self._env.resetWorld()
        self._total_reward = 0
        
        s_prime = self._env.get_observation()
        s_prime = self.discretise_observation(s_prime)
        
        while (not self._env.isEnd()):
            s = s_prime
            R = self.move(s)
            s_prime = self._env.get_observation()
            s_prime = self.discretise_observation(s_prime)
        return self._total_reward

    def update_q_table(self,s: Observation, R: float, s_prime: Observation):
        Q_S_A = self.__q_table[(s,self.__pi_table[s])]
        Q_S_A = Q_S_A + self.__learning_rate * \
                (R + self.__discounted_reward*self.Q(s_prime, self.argmax_a_Q(s_prime,self.__actions)) - Q_S_A)
        
        self.__q_table[(s,self.__pi_table[s])] = Q_S_A

    def Q(self, state: Observation, action: int) -> float:
        if ((state,action) in self.__q_table):
            return self.__q_table[(state,action)]
        else:
            self.__q_table[(state,action)] = 0
            return 0
    
    def argmax_a_Q(self, state: Observation, action_set: List[int]) -> int:
        """Returns action that maximises Q function

        Args:
            state (Observation): state observed
            action_set (List[int]): list of actions possible

        Returns:
            int: action
        """
        return max([(action,self.Q(state,action)) for action in action_set],key=lambda item:item[1])[0]
       
    # todo: action set should be passed into the function for consistency 
    def get_random_action(self) -> int:
        """Randomly generates an action.
        
        Returns:
            int: Action taken
        """
        val = np.random.rand()
        if (float(val) % 1) >= 0.5:
            return math.ceil(val)
        else:
            return round(val)
    
    def print_q_table(self):
        print(self.__q_table)
        
    def get_q_table(self):
        return self.__q_table
    
    def get_pi_table(self):
        return self.__pi_table
        
    def load_pickle(self, parameters_file: str):
        """Loads pickle file to agent table

        Args:
            parameters (str): pickle file location
        """
        if os.path.exists(parameters_file):
            with open(parameters_file, 'rb') as file:
                # Call load method to deserialze
                self.__pi_table,self.__q_table,self.__epsilon,self.__learning_rate = pickle.load(file)
        else:
            print("*** LOG: Pickle file not found")
            pass
        
    def save_pickle(self, parameters_file: str):
        """Saves q and pi table to pickle.

        Args:
            pi_table_file (str): location of file
            q_table_file (str): location of file
        """
        with open(parameters_file, 'wb') as file:
            # Call load method to deserialze
            pickle.dump([self.__pi_table,self.__q_table,self.__epsilon,self.__learning_rate], file)

qlearning_agent = QLearningAgent(world)

## Explanation of Q-Learning Agent

### <span style="color:red">ToDo:</span>
 
  - Find stuff talking about temporal difference
  - Motivation for using a decaying epsilon rate
  - How does the agent obtain the optimal action
  - How does the agent update the q-table and pi-table

Correctness of implementation

In [ ]:
observation = world.resetWorld()
action = qlearning_agent.get_optimal_action(observation)
print(f"{observation}")
print(f"Chosen Action: {action}")

# Task 2:

Demonstrate the effectiveness of the RL agent. Run for 100 episodes (reset the environment at the beginning of each episode) and plot the cumulative reward against all episodes in Jupyter. Print the average reward over the 100 episodes. The average reward should be larger than 195.

### 2a) Effectiveness of MC Agent

In [ ]:
episode_results = np.asarray([mc_agent.run_single_episode() for _ in range(100)])

plt.plot(episode_results)
plt.title('Cumulative reward for each episode')
plt.ylabel('Cumulative reward')
plt.xlabel('episode')
plt.show()

Print the average reward over the 100 episodes.

In [ ]:
print("Average cumulative reward:", episode_results.mean())
print("Is my agent good enough?", episode_results.mean() > 195)

### 2b) Effectiveness of Q-Learning Agent

In [ ]:
episode_results = np.asarray([qlearning_agent.run_single_episode_production() for _ in range(100)])

plt.plot(episode_results)
plt.title('Cumulative reward for each episode')
plt.ylabel('Cumulative reward')
plt.xlabel('episode')
plt.show()

In [ ]:
print("Average cumulative reward:", episode_results.mean())
print("Is my agent good enough?", episode_results.mean() > 195)

## Observations

### <span style="color:red">ToDo: Write observations</span>.

- MC Agent suck balls
- Q-Learning good

## Conclusions

### <span style="color:red">ToDo: Write conclusions</span>.

- Use Q-Learning 


Attach reference for proof of q-learning converging to the optimum policy.

Hence, by running a sizably large number of episodes, we would expect the q-learning agent to converge to the optimum policy (Appendix A), thus achieving the maximum rewards of 500. 

# Task 3:
Render one episode played by the developed RL agent on Jupyter. Please refer to the sample code link for rendering code

In [ ]:
'''
Run one episode of the q-learning agent here, with render = True
'''

# Task 4:

Format the Jupyter notebook by including step-by-step instruction and explanation, such that the notebook is easy to follow and run (refer to the tutorial section in the sample notebook). Include text explanation to demonstrate the originality of your implementation and your understanding of the code. For example, for each task, explain your approach and analyze the output; if you improve an existing approach, explain your improvements.

# Appendix A: Optimal Solution using Physics

[Tang, Eric. "How to Beat the Cartpole Game in 5 Lines." Towards Data Science, 23 Apr. 2018](https://towardsdatascience.com/how-to-beat-the-cartpole-game-in-5-lines-5ab4e738c93f)

In our research, we came across this article which analyzes the cartpole problem. By identifying the physics behind the problem, they were able to derive the optimal policy as follows:

- When the angle θ is “small”, we want to stabilize θ. This is the same as the Omega Policy.
- When the angle θ is “large”, we want to correct θ, i.e., give an angular acceleration towards the center.

We implement a physics agent using the optimal policy as follows:

In [ ]:
class PhysicsAgent(RLAgent):
    
    def __init__(self, env:CartpoleWorld) -> None:
        super().__init__(env)

    def run_production(self, num_of_episode: int) -> None:
        cumulated_reward = 0
        for _ in range(num_of_episode):
            current_reward = self.run_single_episode_production()
            cumulated_reward += current_reward
        print(f"Mean reward is: {cumulated_reward/num_of_episode}")

    def run_single_episode_production(self) -> int:
        # clear history
        self._env.resetWorld()
        self._total_reward = 0
        while (not self._env.isEnd()):
            ndarry: Observation = self._env.get_observation()
            s = self.wrap_observation(ndarry)
            self.move(s)
        return self._total_reward
    
    
    def get_optimal_action(self, s: Observation) -> int:
        """Reference: https://towardsdatascience.com/how-to-beat-the-cartpole-game-in-5-lines-5ab4e738c93f
            Overrides abstract base class
        """
        theta, w = s[2:4]
        if abs(theta) < 0.03:
            return 0 if w < 0 else 1
        else:
            return 0 if theta < 0 else 1
        
physics_agent = PhysicsAgent(world)

Effectiveness of optimal Physics Agent

In [ ]:
episode_results = np.asarray([physics_agent.run_single_episode_production() for _ in range(100)])

plt.plot(episode_results)
plt.title('Cumulative reward for each episode')
plt.ylabel('Cumulative reward')
plt.xlabel('episode')
plt.show()

Render one episode played by the optimal physics agent on Jupyter

In [ ]:
'''
Run one episode of the physics agent here, with render = True
'''